In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [32]:
forecast = pd.read_csv('wc_forecasts.csv')
forecast[forecast['team'] == 'Argentina'].groupby('forecast_timestamp').head()

,forecast_timestamp,team,group,spi,global_o,global_d,sim_wins,sim_ties,sim_losses,sim_goal_diff,...,group_1,group_2,group_3,group_4,make_round_of_16,make_quarters,make_semis,make_final,win_league,timestamp
0,2022-12-18 17:56:03 UTC,Argentina,C,89.64860,2.83610,0.39397,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,1.00000,2022-12-18 17:56:44 UTC
32,2022-12-17 16:54:45 UTC,Argentina,C,88.85631,2.69895,0.37464,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,0.53027,2022-12-18 17:56:44 UTC
64,2022-12-10 21:01:29 UTC,Argentina,C,87.45777,2.58007,0.39356,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,0.64042,0.36568,2022-12-18 17:56:44 UTC
99,2022-12-06 20:54:12 UTC,Argentina,C,87.32471,2.57620,0.39880,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,0.57583,0.24429,0.13042,2022-12-18 17:56:44 UTC
130,2022-12-02 21:01:10 UTC,Argentina,C,87.98006,2.61284,0.38151,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,0.82673,0.51560,0.24966,0.13119,2022-12-18 17:56:44 UTC
164,2022-11-28 20:59:50 UTC,Argentina,C,86.02625,2.48839,0.42171,1.57821,0.26472,1.15707,1.89454,...,0.57290,0.20115,0.22498,0.00097,0.77405,0.49381,0.29805,0.12681,0.06528,2022-12-18 17:56:44 UTC
198,2022-11-24 20:56:17 UTC,Argentina,C,86.09354,2.57779,0.46600,1.08527,0.55644,1.35829,0.54682,...,0.23508,0.30558,0.22864,0.23070,0.54066,0.31108,0.18949,0.09295,0.04724,2022-12-18 17:56:44 UTC
227,2022-11-16 16:00:55 UTC,Argentina,C,87.20776,2.62755,0.43170,1.83671,0.73199,0.43130,3.16936,...,0.59722,0.24273,0.11295,0.04710,0.83995,0.53091,0.32755,0.15944,0.08476,2022-12-18 17:56:44 UTC


In [31]:
forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Fase                        256 non-null    object 
 1   Equipo                      256 non-null    object 
 2   Grupo                       256 non-null    object 
 3   SPI                         256 non-null    float64
 4   Global_Ofensivo             256 non-null    float64
 5   Global_Defensivo            256 non-null    float64
 6   Simulaciones_Ganadas        256 non-null    float64
 7   Simulaciones_Empatadas      256 non-null    float64
 8   Simulaciones_Perdidas       256 non-null    float64
 9   Diferencia_Goles_Simulados  256 non-null    float64
 10  Goles_Anotados              256 non-null    float64
 11  Goles_Recibidos             256 non-null    float64
 12  Grupo_1                     256 non-null    float64
 13  Grupo_2                     256 non

In [34]:
# Reemplazamos el nombre de las columnas por su traducción a Español para facilitar la lectura
forecast.rename(columns={
    'forecast_timestamp': 'Fase',
    'team': 'Equipo',
    'group': 'Grupo',
    'spi': 'SPI',
    'global_o': 'Global_Ofensivo',
    'global_d': 'Global_Defensivo',
    'sim_wins': 'Simulaciones_Ganadas',
    'sim_ties': 'Simulaciones_Empatadas',
    'sim_losses': 'Simulaciones_Perdidas',
    'sim_goal_diff': 'Diferencia_Goles_Simulados',
    'goals_scored': 'Goles_Anotados',
    'goals_against': 'Goles_Recibidos',
    'group_1': 'Grupo_1',
    'group_2': 'Grupo_2',
    'group_3': 'Grupo_3',
    'group_4': 'Grupo_4',
    'make_round_of_16': 'Clasificar_a_16',
    'make_quarters': 'Clasificar_a_Cuartos',
    'make_semis': 'Clasificar_a_Semifinales',
    'make_final': 'Clasificar_a_Final',
    'win_league': 'Ganar_Liga',
    'timestamp': 'Marca_Tiempo'
}, inplace=True)


In [35]:
# Reemplazamos los valores de forecast_timestamp para que representen las fases de cada partido y no las fechas
forecast['Fase'] = forecast['Fase'].replace({
    '2022-12-18 17:56:03 UTC': 'Post Final',
    '2022-12-17 16:54:45 UTC': 'Pre Final',
    '2022-12-10 21:01:29 UTC': 'Pre Semis',
    '2022-12-06 20:54:12 UTC': 'Pre Cuartos',
    '2022-12-02 21:01:10 UTC': 'Pre Octavos',
    '2022-11-28 20:59:50 UTC': 'Pre 3er partido',
    '2022-11-24 20:56:17 UTC': 'Pre 2do partido',
    '2022-11-16 16:00:55 UTC': 'Pre Inicio Copa',
})

forecast[forecast['Equipo'] == 'Argentina'].groupby('Fase').head()

,Fase,Equipo,Grupo,SPI,Global_Ofensivo,Global_Defensivo,Simulaciones_Ganadas,Simulaciones_Empatadas,Simulaciones_Perdidas,Diferencia_Goles_Simulados,...,Grupo_1,Grupo_2,Grupo_3,Grupo_4,Clasificar_a_16,Clasificar_a_Cuartos,Clasificar_a_Semifinales,Clasificar_a_Final,Ganar_Liga,Marca_Tiempo
0,Post Final,Argentina,C,89.64860,2.83610,0.39397,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,1.00000,2022-12-18 17:56:44 UTC
32,Pre Final,Argentina,C,88.85631,2.69895,0.37464,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,1.00000,0.53027,2022-12-18 17:56:44 UTC
64,Pre Semis,Argentina,C,87.45777,2.58007,0.39356,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,1.00000,0.64042,0.36568,2022-12-18 17:56:44 UTC
99,Pre Cuartos,Argentina,C,87.32471,2.57620,0.39880,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,1.00000,0.57583,0.24429,0.13042,2022-12-18 17:56:44 UTC
130,Pre Octavos,Argentina,C,87.98006,2.61284,0.38151,2.00000,0.00000,1.00000,3.00000,...,1.00000,0.00000,0.00000,0.00000,1.00000,0.82673,0.51560,0.24966,0.13119,2022-12-18 17:56:44 UTC
164,Pre 3er partido,Argentina,C,86.02625,2.48839,0.42171,1.57821,0.26472,1.15707,1.89454,...,0.57290,0.20115,0.22498,0.00097,0.77405,0.49381,0.29805,0.12681,0.06528,2022-12-18 17:56:44 UTC
198,Pre 2do partido,Argentina,C,86.09354,2.57779,0.46600,1.08527,0.55644,1.35829,0.54682,...,0.23508,0.30558,0.22864,0.23070,0.54066,0.31108,0.18949,0.09295,0.04724,2022-12-18 17:56:44 UTC
227,Pre Inicio Copa,Argentina,C,87.20776,2.62755,0.43170,1.83671,0.73199,0.43130,3.16936,...,0.59722,0.24273,0.11295,0.04710,0.83995,0.53091,0.32755,0.15944,0.08476,2022-12-18 17:56:44 UTC


In [ ]:
forecast.head(20)

,Fase,Equipo,Grupo,SPI,Global_Ofensivo,Global_Defensivo,Simulaciones_Ganadas,Simulaciones_Empatadas,Simulaciones_Perdidas,Diferencia_Goles_Simulados,...,Grupo_1,Grupo_2,Grupo_3,Grupo_4,Clasificar_a_16,Clasificar_a_Cuartos,Clasificar_a_Semifinales,Clasificar_a_Final,Ganar_Liga,Marca_Tiempo
0,Post Final,Argentina,C,89.64860,2.83610,0.39397,2.0,0.0,1.0,3.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2022-12-18 17:56:44 UTC
1,Post Final,France,D,88.30043,2.96765,0.54381,2.0,0.0,1.0,3.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2022-12-18 17:56:44 UTC
2,Post Final,Morocco,F,73.16416,1.74313,0.53433,2.0,1.0,0.0,3.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2022-12-18 17:56:44 UTC
3,Post Final,Croatia,F,78.82038,2.20264,0.60290,1.0,2.0,0.0,3.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2022-12-18 17:56:44 UTC
4,Post Final,England,B,87.82131,2.71564,0.44261,2.0,1.0,0.0,7.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-12-18 17:56:44 UTC
5,Post Final,Netherlands,A,83.97533,2.52711,0.54940,2.0,1.0,0.0,4.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-12-18 17:56:44 UTC
6,Post Final,Portugal,H,87.02373,2.78069,0.52262,2.0,0.0,1.0,2.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-12-18 17:56:44 UTC
7,Post Final,Brazil,G,93.18946,3.12200,0.28266,2.0,0.0,1.0,2.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-12-18 17:56:44 UTC
8,Post Final,Japan,E,75.96229,2.04074,0.62372,2.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-12-18 17:56:44 UTC
9,Post Final,Senegal,A,74.43038,1.95825,0.63294,2.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-12-18 17:56:44 UTC
